In [1]:
print('Hi')

Hi


# Creating a parent child table for use in a relatinship diagram

In [7]:
import pandas as pd

In [8]:
# Load your wide-format data
df = pd.read_csv('income_statement_alibaba.csv')

In [9]:
# Convert 'Fiscal Date Ending' to datetime and extract the year
df['Fiscal Date Ending'] = pd.to_datetime(df['Fiscal Date Ending'], errors='coerce')  # Convert to datetime
df['Year'] = df['Fiscal Date Ending'].dt.year  # Extract the year

In [11]:

# Define the hierarchical relationships
relationships = {
    'Total Revenue': ['Cost of Revenue', 'Gross Profit', 'Non-Interest Income', 'Other Non-Operating Income', 'Investment Income Net', 'Net Interest Income'],
    'Gross Profit': ['Operating Income', 'Selling, General and Administrative', 'Research and Development'],
    'Operating Income': ['EBIT', 'Operating Expenses', 'Depreciation', 'Depreciation and Amortization'],
    'EBIT': ['EBITDA', 'Income Before Tax'],
    'Income Before Tax': ['Income Tax Expense'],
    'EBITDA': ['Net Income'],
    'Net Income': ['Net Income from Continuing Operations', 'Comprehensive Income Net of Tax'],
    'Net Interest Income': ['Interest Income', 'Interest Expense']
}

# Create a DataFrame to hold the hierarchical data
hierarchical_data = []

# Recursive function to build hierarchical data
def build_hierarchy(year, parent, level, path):
    if level > 4:  # Limit levels to avoid excessive columns
        return
    path[level] = parent
    for child in relationships.get(parent, []):
        value = df.loc[df['Year'] == year, child].values[0] if child in df.columns else None
        hierarchical_data.append([year] + path + [child, value])
        build_hierarchy(year, child, level + 1, path.copy())

# Build hierarchical data
for year in df['Year'].unique():
    for parent in relationships.keys():
        build_hierarchy(year, parent, 0, [None] * 5)  # Adjust the number of levels here if necessary

# Create DataFrame from hierarchical data
df_hierarchy = pd.DataFrame(hierarchical_data, columns=['Year'] + [f'Level {i}' for i in range(5)] + ['Current Metric', 'Value'])

# Save the transformed data to CSV or Excel
df_hierarchy.to_csv('alibaba_hierarchical_levels_columns.csv', index=False)
# Or to Excel
#df_hierarchy.to_excel('hierarchical_levels_columns.xlsx', index=False)

print(df_hierarchy.head())  # Check the first few rows

   Year        Level 0       Level 1           Level 2 Level 3 Level 4  \
0  2024  Total Revenue          None              None    None    None   
1  2024  Total Revenue          None              None    None    None   
2  2024  Total Revenue  Gross Profit              None    None    None   
3  2024  Total Revenue  Gross Profit  Operating Income    None    None   
4  2024  Total Revenue  Gross Profit  Operating Income    EBIT    None   

     Current Metric         Value  
0   Cost of Revenue  6.079150e+11  
1      Gross Profit  3.332530e+11  
2  Operating Income  1.238710e+11  
3              EBIT  1.238710e+11  
4            EBITDA  1.721030e+11  
